# University Student Dropout Prediction Project

### Introduction
In this project, we aim to develop a machine learning model to predict the likelihood of university students dropping out. The challenge of student dropouts is a critical issue in higher education, impacting both the students' future and the educational institutions' effectiveness. Through predictive modeling, we seek to understand the key factors influencing dropout rates and identify at-risk students early in their academic journey.

Our methodology is informed by the approach used by Niyogisubizo et al. in their two-layer ensemble machine learning model. However, our focus expands to the broader context of university-wide student retention, rather than individual classes.

### Project Objectives:

1. **Data Collection:** Acquire comprehensive and relevant datasets from universities, encompassing various factors like student demographics, academic records, engagement levels, and more.
2. **Data Preprocessing:** Clean and preprocess the data to ensure accuracy and reliability for our predictive analysis.
3. **Exploratory Data Analysis (EDA):** Perform in-depth analysis to uncover trends and insights within the data, guiding our feature selection and modeling approach.
4.  **Development:** Construct a predictive model utilizing a stacking ensemble method. This method will integrate different algorithms such as Random Forest, XGBoost, Gradient Boosting, and Feed-forward Neural Networks, leveraging their combined strengths.
5. **Model Evaluation and Tuning:** Utilize relevant performance metrics to evaluate and refine the model, aiming for enhanced predictive accuracy and robustness.
6. **Interpretation and Reporting:** Interpret the results to provide meaningful insights and recommendations, focusing on strategies to improve student retention rates at the university level.

## 1. Data Collection:

**Sources Include:**
University requested student drop out data

## 2. Data Preprocessing

In [2]:
# Importing essential libraries

import pandas as pd
import numpy as np

In [6]:
url = "https://github.com/kflemming30/Student-Drop-Out-Prediction/raw/main/OIR_Student%20Data%20Request.xlsx"
student_df = pd.read_excel(url)
student_df.head()


,PIDM,Cohort,SEX,Degree,Major 1,1st Year GPA,Dorm,1st Year Retention,College,Total Earned Hours,SAT,Major 2,Advisor
0,1,202109F,M,BS,Mechanical Engineering,2.49,Campion Hall,1,SEC,36,NaN,NaN,1.0
1,2,202109F,M,BS,Biology,3.18,Commuter,1,CAS,47,NaN,NaN,2.0
2,3,202109F,M,BS,Chemistry,2.86,Regis Hall,1,CAS,46,NaN,NaN,3.0
3,4,202109F,M,BS,DSB Undeclared,3.84,Gonzaga Hall,1,DSB,45,1300.0,NaN,4.0
4,5,202109F,M,BS,Management,2.69,Commuter,1,DSB,42,NaN,NaN,5.0
